> **Note:** if you are running this notebook on Google Colab, the next cell will install quantumsymmetry and its dependencies:

In [ ]:
%%capture
if 'google.colab' in str(get_ipython()):
    !pip -q install quantumsymmetry

# Encoding fermionic operators

On `quantumsymmetry` we can create an encoding object to store information about the symmetry-adapted encoding for the molecule we are interested in, by using the `Encoding` object: this takes as arguments the molecular geometry (`atom`) and the basis set (`basis`), and as optional arguments the total charge (`charge`), the number of unpaired electrons for open-shell systems (`spin`) and the representation of interest (`irrep`).

Once we have created an `Encoding`, we can use its method `apply` on a fermionic operator object or a Jordan-Wigner qubit operator object in order to convert it into the corresponding qubit operator object in the symmetry adapted encoding. Importantly, if we are trying to encode a product of operators, we'll need to encode it all in a single call of `apply`, as the encoding does not preserve the products of operators (however, you can encode a sum of operators by encoding each term in the sum individually, as the encoding preserves sums).

This might be useful, for instance, when constructing an ansatz circuit such as the [unitary coupled clusters](https://iopscience.iop.org/article/10.1088/2058-9565/aad3e4/meta) (UCC) circuit in a symmetry-adapted encoding.

An example of how to use `Encoding` and `apply` is shown below. It shows that the occupancy operators for the $\sigma_{g \uparrow}$ and the  $\sigma_{u \uparrow}$ spin-orbitals $\hat{N}_{\sigma_{g \uparrow}} \rightarrow \hat{N}_0 = \hat{a}^{\dagger}_0 \hat{a}_0$ and $\hat{N}_{\sigma_{u \uparrow}} \rightarrow \hat{N}_2 = \hat{a}^{\dagger}_2 \hat{a}_2$ in the Jordan-Wigner basis are mapped respectively to the qubit operators $\hat{N}_{\sigma_{g \uparrow}} \rightarrow \frac{I + Z}{2}$ and $\hat{N}_{\sigma_{u \uparrow}} \rightarrow \frac{I - Z}{2}$ in the symmetry-adapted encoding for hydrogen in the minimal basis:


In [ ]:
from quantumsymmetry import Encoding
from openfermion import FermionOperator

encoding = Encoding(atom = 'H 0 0 0; H 0.7414 0 0', basis = 'sto-3g')
fermionic_operator = FermionOperator('0^ 0')
qubit_operator = encoding.apply(fermionic_operator)
print(qubit_operator)

(0.5+0j) [] +
(0.5+0j) [Z0]


In [ ]:
fermionic_operator = FermionOperator('2^ 2')
qubit_operator = encoding.apply(fermionic_operator)
print(qubit_operator)

(0.5+0j) [] +
(-0.5+0j) [Z0]


This is what we would expect: when the lone qubit in the symmetry-adapted encoding for the hydrogen atom is in state $|0⟩$ the orbital $\sigma_{g \uparrow}$ is occupied and the orbital $\sigma_{u \uparrow}$ is unoccupied, and this corresponds to eigenvalues of $1$ for $\hat{N}_{\sigma_{g \uparrow}}$ and $0$ for $\hat{N}_{\sigma_{u \uparrow}}$. The other way round when the qubit in the symmetry-adapted encoding is set to $|1⟩$. It is easy to check that the mappings $\hat{N}_{\sigma_{g \uparrow}} \rightarrow \frac{I + Z}{2}$ and $\hat{N}_{\sigma_{u \uparrow}} \rightarrow \frac{I - Z}{2}$ achieve exactly this.

 Another example shows that the Hermitian double excitation operator $\hat{a}^{\dagger}_0 \hat{a}^{\dagger}_1 \hat{a}_3 \hat{a}_2 + \hat{a}^{\dagger}_2 \hat{a}^{\dagger}_3 \hat{a}_1 \hat{a}_0$ is mapped by the same symmetry-adapted encoding to the qubit operator $X$:

In [ ]:
fermionic_operator = FermionOperator('0^ 1^ 3 2') + FermionOperator('2^ 3^ 1 0')
qubit_operator = encoding.apply(fermionic_operator)
print(qubit_operator)

(1+0j) [X0]


This also makes sense, as the operator $\hat{a}^{\dagger}_0 \hat{a}^{\dagger}_1 \hat{a}_3 \hat{a}_2 + \hat{a}^{\dagger}_2 \hat{a}^{\dagger}_3 \hat{a}_1 \hat{a}_0$ exchanges states $|\sigma_{g \downarrow}\sigma_{g \uparrow}⟩$ and $|\sigma_{u \downarrow}\sigma_{u \uparrow}⟩$, and the Pauli $X$ operator exchanges qubits $|0>$ and $|1>$.

However, an operator that maps states that are in the right symmetry sector only to states that are outside of it, such as the the Hermitian single excitation operator $\hat{a}^{\dagger}_3 \hat{a}_2 + \hat{a}^{\dagger}_2 \hat{a}_3$\, will be projected to the 0 operator:

In [ ]:
fermionic_operator = FermionOperator('3^ 2') + FermionOperator('2^ 3')
qubit_operator = encoding.apply(fermionic_operator)
print(qubit_operator)

0


<p style="text-align: left"> <a href="03_molecular_hamiltonians.ipynb" />< Previous: Building symmetry-adapted molecular Hamiltonians in QuantumSymmetry</a> </p>
<p style="text-align: right"> <a href="05_VQE_circuits.ipynb" />Next: Running a variational algorithm with a symmetry-adapted encoding></a> </p>